In [ ]:
!pip install pylatexenc
!pip install qiskit
!pip install qiskit_aer
!pip install qiskit_ibm_runtime

In [ ]:
import sys

from math import gcd

from qiskit_aer import AerSimulator
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.visualization import plot_histogram
from qiskit_ibm_runtime import SamplerV2 as Sampler

In [ ]:
!git clone https://github.com/jeanfredericlaprade/shor_qiskit.git

sys.path.insert(0,'/content/shor_qiskit')

from Qshor import Shor

Le nombre à factoriser est $15$ et
on utilise un simulateur d'ordinateur quantique fournit par le module `Aer`. 

$15$ est le plus petit nombre que l'algorithme de Shor peut factoriser.

In [ ]:
N = 15 
backend = AerSimulator()

shor = Shor(backend=backend)
result = shor.factor(N)

print(f"Les facteurs de {N} calculés par l'algorithme de Shor sont {result.factors[0][0]} et {result.factors[0][1]}.")

La prochaine cellule calcule le circuit quantique et l'affiche dans une version "haut niveau".<br>
On y voit:
* le registre _up_, qu'on va mesurer pour obtenir un ratio de la fréquence recherchée,
* le registre _down_, dans lequel $a^x\,\text{mod}\,N$ est calculé,
* le registre auxiliaire qui sert de stockage temporaire pour les calculs dans le registre _down_.

Ce circuit correspond à l'algorithme d'évaluation de la phase appliqué à une implémentation efficace de l'opérateur calculant $a^x\,\text{mod}\,N$.

Deux sujet plus avancés (et pour une autre fois) !

In [ ]:
a = 13
circuit = shor.construct_circuit(N, a, measurement=True)

print("Le circuit a une profondeur de", circuit.decompose(reps=4).depth(), "opérateurs de base.")
circuit.draw('mpl', scale=0.7)

In [ ]:

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
sampler = Sampler(backend)
isa_circuit = pm.run(circuit)

result = sampler.run([isa_circuit]).result()

In [ ]:
counts = result[0].data.m.get_counts()
plot_histogram(counts)

Les variations de probabilité qu'on peut observer sont dues au faible nombre d'échantillons que nous avons pris.<br>
Les 4 états mesurés sont en fait équiprobables.
La quantité qui nous interesse est $r$ dans la formule

$\Large \frac{m}{2^n} = \frac{x}{r}$

où:
> $m$ est le résultat de la mesure<br>
> $x$ est un nombre aléatoire<br>
> $n$ est le nombre de qubits dans le registre servant à l'estimation de phase ($8$ dans notre cas).

Pour convertir les valeurs binaires en valeurs décimales, on utilise la convention petit-boutisme (_little endian_):

> si on a la représentation binaire $m = b_{n-1} b_{n-2} \ldots b_1 b_0$, alors
$m = \sum_{i=0}^{n-1} b_i*2^i$.

Par exemple:

> $10010 = 2^1 + 2^4 = 18$

et pour l'état de base $01000000$:

> $\frac{m}{2^n} = \frac{2^6}{2^8} = \frac{1}{4}$

On trouve donc que les quatre valeurs possibles de $\tfrac{x}{r}$ sont $0$, $0.25$, $0.5$ et $0.75$.

$r$ est le dénominateur commun à tous ces nombres.

Déterminez la valeur de r, et à partir de celle-ci déterminez un facteur de $15$

In [ ]:
r = 0  # Mettre la bonne valeur ici!

# a^(r/2) mod N
y = pow(a, r//2, N)

k = gcd(y - 1, N)

print(k)

__Exercice__ 

Imprimez le circuit de l'algorithm de Shor pour plusieurs valeurs de $N$ et déterminez le nombre de qubits nécssaires à l'algorithme en fonction du nombre de bits utilisés par $N$.


In [ ]:
N = 15  # Modifiez cette valeur, par exemple avec N = 21 ou N = 35
nbit = N.bit_length()
print(f'Nombre de bits nécessaires pour encoder {N}: {nbit}')

circuit = shor.construct_circuit(N, 2)
print(f"Nombre de qubits dans le cicuit de l'algorithme de Shor: {circuit.num_qubits}")

circuit.draw('mpl', scale=0.7)